In [1]:
documents = ["bmad_doc/bmad-manual.pdf"]

In [2]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

# Example: Load all PDFs from a directory
loader = DirectoryLoader(
    "bmad_doc",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)
documents = loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,  # Adjust based on your needs
    chunk_overlap=50
)
texts = text_splitter.split_documents(documents)

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # Lightweight local model
)

C:\Users\wdong\AppData\Local\Temp\ipykernel_18336\1082236102.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\wdong\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [ ]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(texts, embeddings)
vector_store.save_local("faiss_index")  # Save for later use